<a href="https://colab.research.google.com/github/YasinEnigma/mci-chatbot/blob/main/task-005/voice_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 6.4 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.7.0-py3-none-any.whl size=192826 sha256=8a0ff1f385ad6cc575c51f9143cced1d9f096836bf9fd3128efc046d2a3a525f
  Stored in directory: /root/.cache/pip/wheels/ba/13/da/8abf941f7cf9f993cde6118a56a7c24e12ed791507acd8ea39
Successfully built pyTelegramBotAPI


In [1]:
!pip install vosk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.2 MB 7.2 MB/s 
     |████████████████████████████████| 112 kB 61.9 MB/s 
  Created wheel for srt: filename=srt-3.5.2-py3-none-any.whl size=22487 sha256=1e35317be10f95eea3852d020960843f3312ffa0ad356a56eb19481a729dc380
  Stored in directory: /root/.cache/pip/wheels/54/c4/ec/4604122e072aebb16803c8297b7cd3f4c72073a3ee58738015
Successfully built srt


In [60]:
#downloading vosk models:
!wget -q -O /content/vosk_model.zip https://alphacephei.com/vosk/models/vosk-model-fa-0.5.zip

In [61]:
!unzip vosk_model.zip

Archive:  vosk_model.zip
   creating: vosk-model-fa-0.5/
   creating: vosk-model-fa-0.5/rescore/
  inflating: vosk-model-fa-0.5/rescore/G.carpa  
  inflating: vosk-model-fa-0.5/rescore/G.fst  
   creating: vosk-model-fa-0.5/test/
 extracting: vosk-model-fa-0.5/test/test.txt  
  inflating: vosk-model-fa-0.5/test/test.wav  
  inflating: vosk-model-fa-0.5/test/test_simple.py  
   creating: vosk-model-fa-0.5/am/
  inflating: vosk-model-fa-0.5/am/final.mdl  
  inflating: vosk-model-fa-0.5/am/tree  
   creating: vosk-model-fa-0.5/graph/
  inflating: vosk-model-fa-0.5/graph/words.txt  
  inflating: vosk-model-fa-0.5/graph/HCLG.fst  
   creating: vosk-model-fa-0.5/graph/phones/
  inflating: vosk-model-fa-0.5/graph/phones/word_boundary.int  
   creating: vosk-model-fa-0.5/conf/
  inflating: vosk-model-fa-0.5/conf/model.conf  
  inflating: vosk-model-fa-0.5/conf/mfcc.conf  
   creating: vosk-model-fa-0.5/ivector/
  inflating: vosk-model-fa-0.5/ivector/splice.conf  
  inflating: vosk-model-fa-0.5

In [62]:
from vosk import Model, KaldiRecognizer
import wave
import json
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read 


import soundfile as sf
import json
import IPython
import librosa
import telebot
import requests
import warnings
warnings.filterwarnings('ignore')

In [63]:
model_path='/content/vosk-model-fa-0.5'

model = Model(model_path)

In [64]:
API_TOKEN = '5784871336:AAFSFwiHl4yRPNFqtg_lxEQVmdR2nRjTqSQ'

In [71]:
bot = telebot.TeleBot(API_TOKEN)

# Handle '/start' and '/help'
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    bot.reply_to(message, """\
Hi there, I am VoiceBot.
I am here to recieve your voices and return transcript back to you. Just say anything nice :)) \
""")

# Handle all other messages with content_type 'text' (content_types defaults to ['text'])
@bot.message_handler(func=lambda message: True)
def echo_message(message):
    bot.reply_to(message, message.text)

@bot.message_handler(content_types=['voice'])
def read_message(message):
	file_info = bot.get_file(message.voice.file_id)
	file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path))
	with open('voice.ogg','wb') as f:
		f.write(file.content)
	x,_ = librosa.load('voice.ogg', sr=16000)
	sf.write('voice.wav', x, 16000)
	bot.send_message(message.chat.id, 'File is Recieved!')
 
	wf = wave.open('voice.wav', 'rb')
	rec = KaldiRecognizer(model, wf.getframerate())
	rec.SetWords(True)
	rec.SetPartialWords(True)
	while True:
		data = wf.readframes(2000)
		if len(data) == 0:
			break
		if rec.AcceptWaveform(data):
			rec.Result()
		else:
			rec.PartialResult()
	res = json.loads(rec.FinalResult())
	bot.reply_to(message, f'Text is: \n {res["text"]}')

In [72]:
if __name__ == '__main__':
    bot.polling(none_stop=True)


bot.infinity_polling()

2022-09-26 06:21:40,719 (__init__.py:886 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
